In [1]:

import time
import traceback
import datetime
import requests
import urllib
import json
import os
import glob
import pathlib
import pandas as pd
import numpy as np
# import openpyxl
# os.chdir("/tf/cc/mojing")
print(os.getcwd())
pd.options.display.max_rows = 999

/home/cc/code/weightunit


# 检查测试用例中的准确率

In [8]:
file_path = "./测试用例.xlsx"
df = pd.read_excel(file_path, engine="openpyxl")
import weightunit as unitparser
from weightunit import parse_str, UnitLevel, default_mu_merge, SingleUnit, MultiUnit

def calc_weight(rows):
    title = rows["title"]

    try:
        ul = UnitLevel.gen_default()
        # ul.fix_unit_level()
        mu1 = parse_str(title, unit_level=ul)
        return mu1.calc_weight()
    except Exception as e:
        traceback.print_exc()
        return '空'

input_str before modifier= 纯牛奶1L*12/24盒2件包邮
input_str after modifier= 纯牛奶1l*12/24盒包邮
unit_list= [(number=1, unit=l,level=1,total_flag=0,start_index=3,end_index=5), (number=12, unit=,level=0,total_flag=0,start_index=6,end_index=8)] 
 op_list= [(op=*,start=5,end=6)]
MUltiUnit.parser_from_str= [1l*12]
final the one multi unit: 1l*12


In [12]:
unitparser.DEBUG = False
df["total-weight"].fillna('空', inplace=True)
df["fill_weight"] = df.apply(lambda row: calc_weight(row), axis=1)
df_miss = df[df["fill_weight"]!= df["total-weight"]]
print('准确率= '+str((1-len(df_miss)/len(df))*100)+'%')
display(df_miss.head(99))

准确率= 99.90118577075098%


,title,total-weight,fill_weight
449,纯牛奶1L*12/24盒2件包邮,24000,12000.0


# 一些使用例子

In [38]:
from weightunit import parse_str, UnitLevel

# 乘法情况, 将升转化为毫升
mu = parse_str("德国进口德亚全脂纯牛奶1L*12盒整箱营养早餐奶学生儿童老人")
print(mu)
print((mu.start_index, mu.end_index))
print(mu.calc_weight())# 计算单位，并转换为ml
print("---------------------")

# 有加法的情况
mu = parse_str("简醇0蔗糖250g*1瓶+开啡尔草莓味200g*5瓶")
print(mu)
print((mu.start_index, mu.end_index))
print(mu.calc_weight())
print("---------------------")

# 或的情况
mu = parse_str("德国进口德亚全脂纯牛奶1L*12盒|24盒整箱营养早餐奶学生儿童老人")
print(mu)
print((mu.start_index, mu.end_index))
print(mu.calc_weight())# 计算单位，并转换为ml
print("---------------------")

# 添加自定义的重量单位
mu = parse_str("手拉葫芦吊机手动2吨倒链1吨10t小型5吨家用起重机3吨工业吊葫芦")
print("未添加【吨】",mu)
ul = UnitLevel.gen_default()
ul.add_unit_word("吨", level=1)
ul.add_unit_word("t", level=1)
mu = parse_str("手拉葫芦吊机手动2吨倒链1吨10t小型5吨家用起重机3吨工业吊葫芦", unit_level=ul)
print("添加【吨】",mu)
print("---------------------")


# 设置str_modifier函数，可以在真正解析前字符串前，删除一些字符，让长串的计算公式连一起
def str_modifier(input_str):
    # 替代默认的str修改，不对input_str进行修改
    return input_str
mu = parse_str("爱尔培 奶酪500g*6盒11月29到期", )
print(mu)
mu = parse_str("爱尔培 奶酪500g*6盒11月29到期", str_modifier=str_modifier)
print(mu)
print("---------------------")

# 获得所有的段落，不强制合并到一个公式里
mu_list = parse_str("3箱爱尔培 奶酪500g*6盒11月29到期", only_one=False)
print(type(mu_list))
print(mu_list)
mu_list = parse_str("3箱爱尔培 奶酪500g*6盒11月29到期", only_one=True)
print(type(mu_list))
print(mu_list)
print("---------------------")

1l*12盒
(11, 17)
12000
---------------------
250g*1瓶+200g*5瓶
(5, 20)
1250
---------------------
1l*(12盒|24盒)
(11, 21)
24000.0
---------------------
未添加【吨】 None
添加【吨】 (2吨)|(1吨)|(10t)|(5吨)|(3吨)
---------------------
500g*6盒
500g*6盒*11
---------------------
<class 'list'>
[3箱, 500g*6盒]
<class 'weightunit.MultiUnit'>
500g*6盒*3箱
---------------------
